In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

In [8]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
train_df.location.value_counts(dropna=False,normalize=True)

NaN                       0.332720
USA                       0.013661
New York                  0.009326
United States             0.006568
London                    0.005911
                            ...   
Quito, Ecuador.           0.000131
In a crazy genius mind    0.000131
Halton Region             0.000131
ÌÏT: 0.0,0.0              0.000131
St Austell, Cornwall      0.000131
Name: location, Length: 3342, dtype: float64

In [4]:
train_df.keyword.value_counts(dropna=False,normalize=True)

NaN                      0.008013
fatalities               0.005911
deluge                   0.005517
armageddon               0.005517
body%20bags              0.005386
                           ...   
forest%20fire            0.002496
epicentre                0.001576
threat                   0.001445
inundation               0.001314
radiation%20emergency    0.001182
Name: keyword, Length: 222, dtype: float64

In [5]:
import nltk  

## remove stop words

In [9]:
from nltk.tokenize import word_tokenize

In [12]:
train_df["tokens"] = train_df.text.apply(lambda x: word_tokenize(x))

In [6]:
from nltk.corpus import stopwords

In [27]:
def remove_stopwords(words):
    return [word.lower() for word in words if word.lower() not in stopwords.words('english')]

In [28]:
train_df["tokens_clean"] = train_df["tokens"].apply(lambda x: remove_stopwords(x))

In [29]:
train_df

,id,keyword,location,text,target,tokens,tokens_clean
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[Our, Deeds, are, the, Reason, of, this, #, ea...","[deeds, reason, #, earthquake, may, allah, for..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[Forest, fire, near, La, Ronge, Sask, ., Canada]","[forest, fire, near, la, ronge, sask, ., canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[All, residents, asked, to, 'shelter, in, plac...","[residents, asked, 'shelter, place, ', notifie..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[13,000, people, receive, #, wildfires, evacua...","[13,000, people, receive, #, wildfires, evacua..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[Just, got, sent, this, photo, from, Ruby, #, ...","[got, sent, photo, ruby, #, alaska, smoke, #, ..."
...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,"[Two, giant, cranes, holding, a, bridge, colla...","[two, giant, cranes, holding, bridge, collapse..."
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,"[@, aria_ahrary, @, TheTawniest, The, out, of,...","[@, aria_ahrary, @, thetawniest, control, wild..."
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,"[M1.94, [, 01:04, UTC, ], ?, 5km, S, of, Volca...","[m1.94, [, 01:04, utc, ], ?, 5km, volcano, haw..."
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,"[Police, investigating, after, an, e-bike, col...","[police, investigating, e-bike, collided, car,..."


In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()